<a href="https://colab.research.google.com/github/zhe0/prac/blob/main/simple_rag_by_gemma_gguf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 快速建置你的機器人
* 對話機器人
    * google gemma
    * 號稱手機地端版
---
* 怎麼減少幻覺
    * 給予參考資料
---
* RAG機制的機器人
    * 給予更大量的參考資料

## 載入套件、下載模組

In [2]:
from langchain.llms import LlamaCpp
from huggingface_hub import hf_hub_download
from IPython.display import HTML
model_name='lmstudio-ai/gemma-2b-it-GGUF'
filename = "gemma-2b-it-q8_0.gguf"
model_path = hf_hub_download(repo_id=model_name,filename=filename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [29]:
llm = None
p={
    'model_path':model_path,
    'temperature':0,
    'max_tokens':8192,
    # 'n_batch':512,# Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    'top_p':.99,
    # 'n_threads':-1, # CPU cores
    'n_ctx':8192,# Context window
    'verbose':False}
llm = LlamaCpp(**p)
llm

## 試著問看看?

In [86]:
q1='FUBON LIFE'
query=f"請問'{q1}是甚麼?"
res=llm.invoke(query)
HTML(res)

In [87]:
context='FUBON可能與金融業相關'
q2='富邦人壽'
prompt = f"""
SYS: 你身處台灣並講著當地語言
ROLE: 你是一位解說員
QUESTION: 請問'{q1}'或'{q2}'是甚麼?
CONTEXT: {context}
THINK: 回答之前，請使用TOT(tree of thought)拆解
ANSWER: 詳盡回答問題
"""
res=llm.invoke(prompt)
HTML(res)

# 建置特定領域知識的機器人

## 先取得關於富邦人壽的相關文件
* 利用wiki完成搜尋

In [31]:
from langchain_community.retrievers import WikipediaRetriever
from langchain.chains import ConversationalRetrievalChain
from IPython.display import Markdown as md
retriever = WikipediaRetriever(lang='zh')
q1='FUBON LIFE'
q2='富邦人壽'
docs = retriever.get_relevant_documents(query=[q1,q2])
ref=''
for i in range(len(docs)):
    ref+=docs[i].metadata['summary']
    print(f'文件{i+1}:',docs[i].metadata['summary'])

富邦人壽保險股份有限公司，簡稱富邦人壽、富壽，是一家總部設於中華民國臺北市松山區的人壽保險公司，為富邦金融控股公司（富邦金控）旗下子公司之一。
富邦人壽大樓是富邦人壽一座已啟用的商辦摩天大樓，位於臺灣臺北市信義區信義計畫區松高路與松勇路之間，土地面積約18,020平方公尺，約佔地 5,451 坪。樓高265.78公尺，地上樓層54樓，地下樓4樓，總樓地板面積為132,362.85平方公尺。包括三棟建築物，分別為辦公塔樓棟、商業設施棟及美術館。由義大利建築師伦佐·皮亚诺（Renzo Piano）與台灣建築師姚仁喜共同設計，目前為台北第4高樓，僅次於臺北101、臺北南山廣場、台北天空塔，臺灣第6高樓。
原本大樓規劃為商場、旅館與商辦三合一的複合式開發案，將引進國際頂級飯店瑰麗酒店，2018年富邦人壽在考量觀光景氣不佳的情況下，將原規畫為飯店的1至19樓空間全數變更為辦公室，預計將會將旗下子公司悉數統一集中至富邦 A25 。
另外原位於松山路的台北富邦銀行松山分行於2023年10月16日遷入本棟地下一樓
富邦凹仔底是富邦人壽一座興建中的摩天大樓，位於臺灣高雄市鼓山區博愛二路與大順一路間，大型地上權開發案，基地原為舊龍華國小 ，2018年3月7日開標由富邦人壽得標，5月10日完成簽約。 開發案土地面積約11,636坪，將打造一座融合商場、旅館、辦公室及水族館的商業綜合體，建築設計由三菱地所設計負責，將為一座地下6層地上47層的建築物。2020年12月3日富邦集團與三地集團簽約，將開發案中的水族館交由三地集團旗下的南仁湖企業經營，未來將會是高雄第三高樓。2022年3月23日補辦動土典禮，2023年2月確定商場將由漢神百貨進駐，成為高雄市第三間漢神百貨購物中心。柏文健康事業 （页面存档备份，存于互联网档案馆）向富邦人壽承租20年，總面積達7,000坪之商場空間，未來旗下「健身工廠」、「人體工房」、「滾吧」、「肖跳」、「鐳射戰場」及「S klub兒童體適能俱樂部」六大事業品牌將同時進駐，且再打造南台灣唯一符合國際標準之滑冰場，此一滑冰場為南臺灣最大冰宮，未來可兼具娛樂、休閒，又可舉辦國際冰上曲棍球賽事。


## 基於wiki紀錄，或是改變指令，對話看看?

In [98]:
prompt = f"""
Prerequisite: 你身處台灣並講著當地語言
role: 你是一位解說員
Question: 請問'{q1}'或'{q2}'是甚麼?
context: {ref}
think: 回答之前，請使用TOT(tree of thought)拆解
answer: 詳盡回答TOT步驟，並再次回答
temperature=1.0
"""
res=llm.invoke(prompt)
md(res)

TOT:

**FUBON LIFE**

* 富邦人壽保險股份有限公司，簡稱富邦人壽、富壽，是一家總部設於中華民國臺北市松山區的人壽保險公司。

**富邦人壽大樓**

* 富邦人壽一座已啟用的商辦摩天大樓，位於臺灣臺北市信義區信義計畫區松高路與松勇路之間，土地面積約18,020平方公尺，約佔地 5,451 坪。
*樓高265.78公尺，地上樓層54樓，地下樓4樓，總樓地板面積為132,362.85平方公尺。
*包括三棟建築物，分別為辦公塔樓棟、商業設施棟及美術館。

In [97]:
q1='FUBON LIFE'
q2='富邦人壽'
context=ref

prompt = f"""
Prerequisite: 你身處台灣並講著當地語言
role: 你是一位解說員
Question: 請問'{q1}'或'{q2}'是甚麼，並且使用SWOT方式解說?
context: {ref}
think: 回答之前，請使用TOT(tree of thought)拆解
answer: 詳盡回答TOT步驟，並再次回答
temperature=1.0
"""
res=llm.invoke(prompt)
md(res)

# TOT (tree of thought)

**FUBON LIFE**

* FUBON LIFE 是台灣一家總部設於臺北市的**人壽保險公司**。
* 公司主要業務為提供**個人壽險**、**家庭保險**和**企業保險**等產品和服務。

**SWOT 分析**

**優勢 (Strengths)**

* **豐富的產品和服務：** FUBON LIFE 提供多種產品和服務，可以滿足不同顧客的需求。
* **強大的品牌知名度：** FUBON LIFE 是台灣知名人壽保險公司的品牌，可以為其帶來良好的品牌知名度。
* **多元的業務範圍：** FUBON LIFE 的業務範圍很廣，可以為顧客提供多種選擇。

**劣勢 (Weaknesses)**

* **競爭激烈：** 台灣人壽保險市場非常激烈，FUBON LIFE 需要不斷努力以保持其競爭力。
* **人力資源有限：** FUBON LIFE 的人力資源有限，可能影響其營運效率。
* **市場反應緩慢：** 台灣的經濟發展緩慢，可能影響 FUBON LIFE 的營運效率。

**機會 (Opportunities)**

* **市場成長：** 台灣的經濟發展將推動人壽保險的市場成長。
* **新技術：** 新技術可以幫助 FUBON LIFE 提高其營運效率。
* **國際化：** FUBON LIFE 可以透過國際化來擴展其市場。

**威脅 (Threats)**

* **經濟衰退：** 經濟衰退可能會導致人壽保險公司的營運困難。
* **競爭激烈：** 台灣人壽保險市場非常激烈，FUBON LIFE 需要不斷努力以保持其競爭力。
* **法律規範：** 人壽保險公司必須遵守嚴格的法律規範。

# 建置 金管會的AI管理辦法 機器人

## 下載
* 金管會的AI管理辦法的pdf資料
* 下載用來產生embedding的模組

In [40]:
import requests,glob

url = 'https://www.fsc.gov.tw/uploaddowndoc?file=news/202310171518250.pdf&filedisplay=%E9%99%84%E4%BB%B6_%E9%87%91%E8%9E%8D%E6%A5%AD%E9%81%8B%E7%94%A8%E4%BA%BA%E5%B7%A5%E6%99%BA%E6%85%A7%28AI%29%E4%B9%8B%E6%A0%B8%E5%BF%83%E5%8E%9F%E5%89%87%E8%88%87%E7%9B%B8%E9%97%9C%E6%8E%A8%E5%8B%95%E6%94%BF%E7%AD%96.pdf&flag=doc'
res=requests.get(url)
pdf_path = './ai.pdf'
with open(pdf_path,'wb') as file:
  file.write(res.content)

from langchain_community.document_loaders.pdf import PyPDFLoader
pdfs = glob.glob(pdf_path)

pi = 0
loader = PyPDFLoader(pdfs[pi], )
pages = loader.load()
pages[0].page_content

'1 \n 金融業運用 人工智慧 (AI)之核心原則與相關推動 政策  \n一、前言  \n近來 AI1在金融服務 領域的應用日益增加， 為金融產業\n提供客戶服務 帶來效益，亦同時衍生一些新的風險問題 及監\n理挑戰。 為協助金融機構善用 AI科技優勢，並 能有效管理\n風險、確保公平、保護消費者權益、維護系統安全及實現永\n續發展，本會 依據行政院「 臺灣 AI行動計畫 2.0」政策規劃\n及「數位政策法制協調 專案會議」之推動策略 ，並參考全球\n主要國家監理機關及國際組織之相關指導原則， 及結合我國\n金融市場發展狀況及本會監理政策方向，擬定適合我國金融\n業的 6項AI應用核心原則，以 期引導金融業在兼顧消費者\n權益、金融市場秩序及社會責任下，積極投入科技創新，促\n進金融服務升級。  \n以下就AI之影響及國際組織或 主要國家對運用 AI之立\n場與規定、我國金融業運用 AI現況、訂定 AI原則及政策 之\n必要性、我國金融業運用 AI之6項核心原則 ，以及本會因\n應AI發展推動之配套政策等事項進行說明。  \n \n \n1 由於 AI 技術與日俱進，因此各國際組織或各國並未定義 AI，反而聚焦「 AI 系統」，本文交替\n使用 AI與AI系統。依據經濟合作暨發展組織 (OECD)對「AI 系統」之定義，係指一種以機器為\n基礎的系統，在給予設定之一組目的下，能透過產製輸出品 (例如預測、建議或決策 )來影響環境。\n它使用以機器或人類為基礎的數據與輸入品來 (1)感知真實或虛擬環境； (2)萃取這些感知，並透\n過自動分析 (例如，使用機器學習 )或人工分析，再轉化為模型；以及 (3)使用模型推理來形成結果\n的選項。 AI系統係設計以不同的自主程度來運作。  '

In [75]:
import torch
from langchain.embeddings import HuggingFaceEmbeddings
# model_name="aspire/acge_text_embedding"
emb_model_name='amu/tao-8k'
device='cuda' if torch.cuda.is_available() else 'cpu'
# https://huggingface.co/spaces/mteb/leaderboard #aspire/acge_text_embedding#amu/tao-8k
embeddings = HuggingFaceEmbeddings(
    model_name=emb_model_name,
    model_kwargs={"device": device},
    encode_kwargs={"normalize_embeddings": True},
)
embeddings

config.json:   0%|          | 0.00/871 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/667M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='amu/tao-8k', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, multi_process=False, show_progress=False)

## vector db
>    1. 分割文本(chunk)
>    1. 轉換為embedding
>    1. 快速查找相關文本
>    1. 文本回傳給機器

In [70]:
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
chunk_size=100
text_splitter = RecursiveCharacterTextSplitter(
# text_splitter = CharacterTextSplitter( #經嘗試沒有比較好
    chunk_size=chunk_size,
    # separators=["\s{2}",'[\n\t]{1,}','\r\n'],
    # separator='[\n\t]{1,}',
    chunk_overlap=int(chunk_size/10),
    length_function=len,
    is_separator_regex=True)
texts = text_splitter.split_documents(pages)
print('經過切割共',len(texts),'頁')
print('最後一頁內容: ',texts[-1])

all_texts=' '.join([i.page_content for i in texts])
print('pdf最後100字的內容: ',all_texts[-100:])

經過切割共 189 頁
最後一頁內容:  page_content='當之教育及培訓，使員工能適應 AI帶來之變革，並盡\n可能維護其 工作權益。' metadata={'source': './ai.pdf', 'page': 25}
pdf最後100字的內容:  ，保護自然環境，從而促進包容性成長、永續發
展及社會福祉。  
(二)金融機構在 AI系統運用過程中， 宜對一般員工 提供適 當之教育及培訓，使員工能適應 AI帶來之變革，並盡
可能維護其 工作權益。


In [90]:
docs = text_splitter.create_documents([all_texts])
print('切割後的總頁數',len(docs))
print('第一頁文本: ',docs[0].page_content)
print('最後頁文本: ',docs[-1].page_content)

切割後的總頁數 186
第一頁文本:  1 
 金融業運用 人工智慧 (AI)之核心原則與相關推動 政策  
一、前言  
近來 AI1在金融服務 領域的應用日益增加， 為金融產業
最後頁文本:  展及社會福祉。  
(二)金融機構在 AI系統運用過程中， 宜對一般員工 提供適 當之教育及培訓，使員工能適應 AI帶來之變革，並盡
可能維護其 工作權益。


## 準備問答之前
* 製作vector db
* 製作prompt範本

In [76]:
%%time
from langchain.vectorstores import Chroma
db = Chroma.from_documents(docs, embeddings, persist_directory="db")
db

CPU times: user 2min 48s, sys: 8.77 s, total: 2min 57s
Wall time: 3min 3s


In [94]:
from langchain import PromptTemplate
from langchain.chains.retrieval_qa.base import RetrievalQA
template = """
role: 你是「台灣」的金融監督管理委員會之「窗口」
question: {query}
answer: 回覆之前請檢視自己的答案，不可以捏照回答。「詳細的」回答你的客戶
temperature=0.0
"""

qa_prompt = PromptTemplate(
    input_variables=["query"],
    template=template,
)

p={
    'retriever':db.as_retriever(search_kwargs={'k':10}),
    'return_source_documents':True}
qa_chain = RetrievalQA.from_chain_type(llm,**p)
qa_chain

RetrievalQA(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"), llm=LlamaCpp(verbose=False, client=<llama_cpp.llama.Llama object at 0x79a0c6ac3d30>, model_path='/root/.cache/huggingface/hub/models--lmstudio-ai--gemma-2b-it-GGUF/snapshots/a0b140bfb922a743f89dd0682a24a17516071ab9/gemma-2b-it-q8_0.gguf', n_ctx=8192, max_tokens=8192, temperature=0.0, top_p=0.99)), document_variable_name='context'), return_source_documents=True, retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x799e1645b970>, search_kwargs={'k': 10}))

## rag機制下的機器人，問答看看?

In [93]:
res = qa_chain({"query": '甚麼是SupTech? 金管會是否有何具體的實施建議?'})
print('問題: ',res['query'])
print()
print('機器人回應: ',res['result'])
print()
print('參考出處: ',res['source_documents'])

問題:  甚麼是SupTech? 金管會是否有何具體的實施建議?

機器人回應:   SupTech 是指利用 AI 技術發展的監理科技，以提升金融監理效率及有效性。 金管會沒有具體的實施建議，但建議金融機構積極參與 AI 的研發及應用，以提供客戶更優質的金融服務。

參考出處:  [Document(page_content='(二)持續檢視本會相關規範，並適時進行法規調適，以 建立金\n融業應用 AI系統之完善法規環境 。 (三)利用 AI技術發展監理科技 (SupTech )，以提升金融監理之\n效率及有效性 。'), Document(page_content='可協助金融機構更有效率地處理 訊息，亦可透過法遵科 技(RegTech )及監理科技 (SupTech )，以確保金融機構的 遵\n法性並增強監管效能。同時， FSB亦提醒相關風險如下：'), Document(page_content='更優質之金融服務 或發展法遵科技 (RegTech) ，並將透過 舉辦研討會、工作坊等方式，協助金融機構導入最佳實務\n做法。  \n(六)持續就金融業應用 AI系統之實際情況進行瞭解及檢視，'), Document(page_content='進行溝通對話，並視業界發展情況及應用情形適時 評估 11 \n 調整相關規範，以利金融機構遵循。  \n(二)本會針對金融機構導入新興科技 之相關規範：  \n 1.銀行業：'), Document(page_content='件發生時規劃解決方式。 另業者應有 AI相關運算規則並留\n存軌跡紀錄，俾利後續驗證與管理。 透過「確保系統穩健性 與安全性」這項原則，本會期望\n能在維護金融穩定與保護消費者權益的同時，讓金融機構能'), Document(page_content='關係人提供妥適的透明度與揭露資訊 ；(6)有妥適控制機 制，以確保資料及 AI與ML的表現能 將偏見最小化。  \n美國13、新加坡14、南韓15等國金融監理機關 針對金'), Document(page_content=')12」，提出 6項金融監理'), Document(page_content='金融市場發展狀況及本會監理政策方向，擬定適合我國金融 業的 6項AI應用核心原則，以 期引導金融業在兼顧消費者\n權益、金融市場秩序及社會責任下，積